<a href="https://colab.research.google.com/github/zzhining/stock_market_analysis/blob/main/22_short_term_trading_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 단기 투자 전략

- 방법 1 : 모멘텀 전략
    - 상승세에 올라탄 주식을 사서 더 오르길 기대하는 전략
    - MACD 전략

- 방법 2 : 평균회귀 전략
    - 원래 수준보다 많이 하락한 종목을 사서 원래대로 회복하길 기다리는 평균 회귀 전략
    - 매수와 매도 시그널이 존재함
    - 주가의 움직임이 박스권을 형성하고 있는 종목에 적합

In [ ]:
!pip install finterstellar

In [ ]:
import finterstellar as fs

# MACD

## 개념
- Moving Average Convergence Divergence
- 이동평균수렴확산지수

---
- MA(단순이동평균)
- EMA(지수이동평균)
    - 최근 값에 더 높은 가중치를 주고 계산한 평균값
- MACD
    - 단기 평균에서 장기 평균을 빼준 값
    - MACD가 양수면, 주가가 상승 추세임
- MACD signal
    - MACD의 후행성을 극복하기 위해 만든 선
    - 9일간 MACD 지수 이동 평균선
- MACD Oscillator
    - MACD를 더 쉽게 이해하기 위한 보조지표
    - MACD에서 MACD signal값을 빼 히스토그램 형식으로 표현
    - 0 이상인 경우 매수추천(by 제럴드 아펠)
    

## 투자전략
- 1) MACD가 플러스이면 매수, 마이너스이면 매도하는 전략
- 2) MACD Oscillator가 플러스이면 매수, 마이너스이면 매도하는 전략

In [ ]:
symbol = 'MSFT'
df = fs.get_price(symbol, start_date = '2023-01-01', end_date = '2024-04-09')
fs.draw_chart(df, right=symbol)

In [ ]:
fs.macd(df)

In [ ]:
fs.draw_chart(df, right=['macd', 'macd_signal', 'macd_oscillator'])

In [ ]:
# macd_oscillator가 0이상인 경우 매수하는 시그널 생성
fs.indicator_to_signal(df, factor='macd_oscillator', buy=0, sell=0)

In [ ]:
fs.position(df)
fs.draw_chart(df, right='position_chart', left='macd_oscillator')

In [ ]:
fs.evaluate(df, cost=.001)
fs.performance(df, rf_rate = 0.01)
fs.draw_trade_results(df)

### (참고) macd()

In [ ]:
def macd(df, short=12, long=26, signal=9):
    '''
    Calculate MACD indicators
    :param df: Dataframe containing historical prices
    :param short: Day length of short term MACD
    :param long: Day length of long term MACD
    :param signal: Day length of MACD signal
    :return: Dataframe of MACD values
    '''
    symbol = df.columns[0]
    df['ema_short'] = df[symbol].ewm(span=short).mean()
    df['ema_long'] = df[symbol].ewm(span=long).mean()
    df['macd'] = (df['ema_short'] - df['ema_long']).round(2)
    df['macd_signal'] = df['macd'].ewm(span=signal).mean().round(2)
    df['macd_oscillator'] = (df['macd'] - df['macd_signal']).round(2)
    return df[[symbol, 'macd','macd_signal','macd_oscillator']]

# RSI

## 개념
- Relative Strength Index, [[지표설명](https://www.investopedia.com/terms/r/rsi.asp)]
- 상대강도지수


## 투자 전략
- RSI가 70 이상이면 과매수(over bought)
    - 주식을 너무 많이 사서 주가가 과열되어 있음
- RSI가 30이하이면 과매도(over sold)
- 웰리스 와일더 전략
    - 과매도 구간에 주식을 사고, 과매수 구간에서 주식을 파는 전략
    - RSI를 구하는 기간은 14일을 기본으로 제시
- 모멘텀 전략
    - 과매수 구간에 주식을 사고, 과매도 구간에 주식을 파는 전략

In [ ]:
symbol = 'VZ'
df = fs.get_price(symbol, start_date = '2023-01-01', end_date='2024-04-09')
fs.draw_chart(df, right=symbol)

In [ ]:
fs.rsi(df, w=14)
fs.draw_chart(df, left='rsi', right=symbol)

In [ ]:
fs.indicator_to_signal(df, factor='rsi', buy=30, sell=70)
fs.position(df)
fs.evaluate(df, cost=.001)
fs.performance(df, rf_rate = 0.01)
fs.draw_trade_results(df)

In [ ]:
fs.indicator_to_signal(df, factor='rsi', buy=70, sell=50)
fs.position(df)
fs.evaluate(df, cost=.001)
fs.performance(df, rf_rate = 0.01)
fs.draw_trade_results(df)

### (참고) rsi()

In [ ]:
def rsi(df, w=14):
    '''
    Calculate RSI indicator
    :param df: Dataframe containing historical prices
    :param w: Window size
    :return: Series of RSI values
    '''
    pd.options.mode.chained_assignment = None
    symbol = df.columns[0]
    df.fillna(method='ffill', inplace=True)  # 들어온 데이터의 구멍을 메꿔준다
    if len(df) > w:
        df['diff'] = df.iloc[:,0].diff()   # 일별 가격차이 계산
        df['au'] = df['diff'].where(df['diff']>0, 0).rolling(w).mean()
        df['ad'] = df['diff'].where(df['diff']<0, 0).rolling(w).mean().abs()
        for r in range(w+1, len(df)):
            df['au'][r] = ( df['au'][r-1]*(w-1) + df['diff'].where(df['diff']>0,0)[r] ) / w
            df['ad'][r] = ( df['ad'][r-1]*(w-1) + df['diff'].where(df['diff']<0,0).abs()[r] ) / w
        df['rsi'] = (df['au'] / (df['au'] + df['ad']) * 100).round(2)
        return df[[symbol, 'rsi']]
    else:
        return None

# 엔벨로프


## 개념
- envelope(주가를 봉투처럼 감싼다)
- 이동평균을 이용한 지표: 이동평균보다 +/-5% 가격을 기준으로 envelope 생성
- 현재 가격이 이동평균 가격보다 얼마나 많이 떨어져 있는지 한 눈에 알아보기 쉬움


## 투자전략
- 차트상에서 현재 가격이 그래프의 어느 영역에 있는지를 확인
    - A구역: 밴드 상단보다 위 구간
    - B구역: 밴드 중심과 상단 사이
    - C구역: 밴드 하단과 중심 사이
    - D구역: 밴드 하단 아래 구간

- 모멘텀투자자 or 성장주
    - A구역: 매수, B구역: 매도
- 평균회귀투자자 or 박스권
    - D구역: 매수, B구역: 매도

In [ ]:
symbol = 'BA'
df = fs.get_price(symbol, start_date = '2023-01-01', end_date='2024-04-09')
fs.draw_chart(df, right=symbol)

In [ ]:
fs.envelope(df, w=20, spread=.1)
df.tail()

In [ ]:
fs.band_to_signal(df, buy='A', sell='B')

In [ ]:
fs.position(df)
fs.evaluate(df, cost=.001)
fs.performance(df, rf_rate = 0.01)
fs.draw_trade_results(df)

In [ ]:
fs.band_to_signal(df, buy='D', sell='B')
fs.position(df)
fs.evaluate(df, cost=.001)
fs.performance(df, rf_rate = 0.01)
fs.draw_trade_results(df)

###(참고) envelope()

In [ ]:
def envelope(df, w=50, spread=.05):
    '''
    Calculate Envelope indicators
    :param df: Dataframe containing historical prices
    :param w: Window size
    :param spread: % difference from center line to determine band width
    :return: Dataframe of Envelope values
    '''
    symbol = df.columns[0]
    df['center'] = df[symbol].rolling(w).mean()
    df['ub'] = df['center']*(1+spread)
    df['lb'] = df['center']*(1-spread)
    return df[[symbol, 'center','ub','lb']]

# 볼린저 밴드

## 개념
- [Bollinger Band](https://namu.wiki/w/%EB%B3%BC%EB%A6%B0%EC%A0%80%20%EB%B0%B4%EB%93%9C)
- 이동평균에 변동성을 결합한 그래프

## 투자 전략
- 모멘트 전략
    - 밴드 상단 매매
- 평균회귀 전략
    - 밴드 하단 매매

- '볼린저밴드 투자기법(존 볼린저)'
    - 볼린저밴드를 모멘텀 전략으로 이용하는 것을 권장
    - 주가가 상승할 때는 밴드의 상단을 타고 올라가고, 주가가 하락할 때는 하단 밴드를 타고 떨어짐

- 인기 지표
    - 기본 세팅 `BB(20, 2)`을 투자 시그널로 이용하는 사람이 너무 많아 투자 기회가 많지 않으므로 다양한 세팅으로 백테스팅 해보는 것이 필요함

In [ ]:
symbol = 'TSM'
df = fs.get_price(symbol, start_date = '2020-01-01', end_date='2024-04-09')
fs.draw_chart(df, right=symbol)

In [ ]:
fs.bollinger(df, w=20, k=2)
df.tail()

In [ ]:
fs.draw_band_chart(df)

In [ ]:
fs.band_to_signal(df, buy='A', sell='B')
fs.position(df)
fs.evaluate(df, cost=.001)
fs.performance(df, rf_rate = 0.01)
fs.draw_trade_results(df)

In [ ]:
fs.band_to_signal(df, buy='D', sell='B')
fs.position(df)
fs.evaluate(df, cost=.001)
fs.performance(df, rf_rate = 0.01)
fs.draw_trade_results(df)

In [ ]:
df = fs.get_price(symbol, start_date = '2023-01-01', end_date='2024-04-09')
fs.bollinger(df, w=20, k=1)
fs.draw_chart(df, right=symbol)

In [ ]:
fs.band_to_signal(df, buy='A', sell='B')
fs.position(df)
fs.evaluate(df, cost=.001)
fs.performance(df, rf_rate = 0.01)
fs.draw_trade_results(df)

In [ ]:
fs.band_to_signal(df, buy='D', sell='B')
fs.position(df)
fs.evaluate(df, cost=.001)
fs.performance(df, rf_rate = 0.01)
fs.draw_trade_results(df)

## 합성 시그널 생성
- 모멘텀 시그널 OR 평균회귀 시그널

In [ ]:
df = fs.get_price(symbol, start_date='2020-01-01', end_date='2024-04-09')
df = fs.bollinger(df, w=20, k=1)
#모멘텀 시그널
df['s1'] = fs.band_to_signal(df, buy='A', sell='B')
#평균회귀 시그널
df['s2'] = fs.band_to_signal(df, buy='D', sell='B')

# 시그널 조합
fs.combine_signal_or(df, 's1', 's2')

In [ ]:
fs.position(df)
fs.evaluate(df, cost=.001)
fs.performance(df, rf_rate = 0.01)
fs.draw_trade_results(df)

### (참고) bollinger

In [ ]:
def bollinger(df, w=20, k=2):
    '''
    Calculate bollinger band indicators
    :param df: Dataframe containing historical prices
    :param w: Window size
    :param k: Multiplier to determine band width
    :return: Dataframe of bollinger band values
    '''
    symbol = df.columns[0]
    df['center'] = df[symbol].rolling(w).mean()
    df['sigma'] = df[symbol].rolling(w).std()
    df['ub'] = df['center'] + k * df['sigma']
    df['lb'] = df['center'] - k * df['sigma']
    return df[[symbol, 'center','ub','lb']]

# 스토캐스틱

## 개념

- Stochastic
- 최근 N일간 주가 범위 중 현재 주가가 얼마나 높이 있는가

## 투자전략
- 평균회귀 전략
    - slow k < 20: 과매도 구간, slow k > 80: 과매수 구간
    - 20보다 낮을 때 사고, 80보다 높을 때 팔아라
- 모멘텀 전략
    - (slow k - slow d) > 0 : 상승추세 --> 매수
    - (slow k - slow d) < 0 : 하락추세 --> 매도

In [ ]:
symbol = 'DAL'
df = fs.get_ohlc(symbol, start_date = '2020-01-01', end_date = '2024-04-09')
df.tail()

In [ ]:
fs.stochastic(df, symbol, n=14, m=3, t=3)

In [ ]:
fs.draw_chart(df, left='slow_k', right=symbol)

In [ ]:
# slow_k가 20보다 낮아지면 매수, 80보다 커지면 매도
fs.indicator_to_signal(df, factor='slow_k', buy=20, sell=80)
fs.position(df)
fs.evaluate(df, cost=.001)
fs.performance(df, rf_rate = 0.01)
fs.draw_trade_results(df)

### 모멘텀 전략

In [ ]:
df = fs.get_ohlc(symbol, start_date = '2020-01-01', end_date = '2024-04-09')
fs.stochastic(df, symbol, n=14, m=3, t=3)
df['indicator'] = df['slow_k'] - df['slow_d']

In [ ]:
df.tail()

In [ ]:
fs.indicator_to_signal(df, factor='indicator', buy=0, sell=0)
fs.position(df)
fs.evaluate(df, cost=.001)
fs.performance(df, rf_rate = 0.01)
fs.draw_trade_results(df)

### (참고) stochastic()

In [ ]:
def stochastic(df, symbol, n=14, m=3, t=3):
    '''
    Calculate stochastic indicators
    :param df: Dataframe containing historical prices
    :param symbol: Symbol or ticker of equity by finance.yahoo.com
    :param n: Day length of fast k stochastic
    :param m: Day length of slow k stochastic
    :param t: Day length of slow d stochastic
    :return: Dataframe of stochastic values
    '''
    try:
        df['fast_k'] = ( ( df['Close'] - df['Low'].rolling(n).min() ) / ( df['High'].rolling(n).max() - df['Low'].rolling(n).min() ) ).round(4) * 100
        df['slow_k'] = df['fast_k'].rolling(m).mean().round(2)
        df['slow_d'] = df['slow_k'].rolling(t).mean().round(2)
        df.rename(columns={'Close':symbol}, inplace=True)
        df.drop(columns=['High','Open','Low','Volume','Adj Close','fast_k'], inplace=True)
        return df[[symbol, 'slow_k', 'slow_d']]
    except:
        return 'Error. The stochastic indicator requires OHLC data and symbol. Try get_ohlc() to retrieve price data.'
